In [28]:
import pandas as pd

df = pd.read_csv('../data/2018-11-greater-manchester-street.csv').dropna(subset=['Longitude', 'Latitude'])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32672 entries, 0 to 32671
Data columns (total 12 columns):
Crime ID                 27732 non-null object
Month                    32672 non-null object
Reported by              32672 non-null object
Falls within             32672 non-null object
Longitude                32672 non-null float64
Latitude                 32672 non-null float64
Location                 32672 non-null object
LSOA code                32672 non-null object
LSOA name                32672 non-null object
Crime type               32672 non-null object
Last outcome category    27732 non-null object
Context                  0 non-null float64
dtypes: float64(3), object(9)
memory usage: 3.2+ MB
None


In [29]:
df['Coordinate'] = df.apply(lambda x: (x['Longitude'], x['Latitude']), axis=1)

df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Coordinate
0,NaN,2018-11,Greater Manchester Police,Greater Manchester Police,-2.437410,53.613882,On or near Radstock Close,E01004768,Bolton 001A,Anti-social behaviour,NaN,NaN,"(-2.43741, 53.613882)"
1,NaN,2018-11,Greater Manchester Police,Greater Manchester Police,-2.464422,53.612495,On or near Parking Area,E01004768,Bolton 001A,Anti-social behaviour,NaN,NaN,"(-2.4644220000000003, 53.612494999999996)"
2,NaN,2018-11,Greater Manchester Police,Greater Manchester Police,-2.464422,53.612495,On or near Parking Area,E01004768,Bolton 001A,Anti-social behaviour,NaN,NaN,"(-2.4644220000000003, 53.612494999999996)"
3,5be2224443986a271775604a58446bb0ed5332ebbeae6a...,2018-11,Greater Manchester Police,Greater Manchester Police,-2.440584,53.613781,On or near Wincanton Drive,E01004768,Bolton 001A,Vehicle crime,Investigation complete; no suspect identified,NaN,"(-2.440584, 53.613781)"
4,a48233f8638de9adcc0835ca4b82768ca3ee527d5b59e4...,2018-11,Greater Manchester Police,Greater Manchester Police,-2.437522,53.613010,On or near Midford Drive,E01004768,Bolton 001A,Vehicle crime,Unable to prosecute suspect,NaN,"(-2.437522, 53.613009999999996)"


In [30]:
crime_sr = df.groupby(['Coordinate', 'Crime type']).size()

In [31]:
crime_freq = df.groupby('Coordinate').size()

print(crime_freq.min())
print(crime_freq.max())

1
158


In [32]:
features = []

last_coord = None
data = None

for (coord, crime_type), crime_count in crime_sr.iteritems():
    
    # Create a new data if it's the new location
    if coord != last_coord:
        if data is not None:
            features.append(data)
                
        # Re-initialise the data
        data = {
            'type': 'Feature',
            'properties': {
                'totalCrimeCount': 0,
                'crimeCountPerType': {}
            },
            'geometry': {
                'type': 'Point',
                'coordinates': []
            }
        }
        
    data['properties']['totalCrimeCount'] += crime_count
    data['properties']['crimeCountPerType'][crime_type] = crime_count
    data['properties']['coordinates'] = f'{coord[1]}, {coord[0]}'
    data['geometry']['coordinates'] = [coord[0], coord[1]]
    
    last_coord = coord

In [33]:
features

[{'geometry': {'coordinates': [-2.718849, 53.5211], 'type': 'Point'},
  'properties': {'coordinates': '53.5211, -2.718849',
   'crimeCountPerType': {'Violence and sexual offences': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-2.716583, 53.521752], 'type': 'Point'},
  'properties': {'coordinates': '53.521752, -2.716583',
   'crimeCountPerType': {'Vehicle crime': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-2.716439, 53.52395500000001], 'type': 'Point'},
  'properties': {'coordinates': '53.52395500000001, -2.716439',
   'crimeCountPerType': {'Other theft': 1, 'Robbery': 1},
   'totalCrimeCount': 2},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-2.716316, 53.533671999999996],
   'type': 'Point'},
  'properties': {'coordinates': '53.533671999999996, -2.716316',
   'crimeCountPerType': {'Violence and sexual offences': 1},
   'totalCrimeCount': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-2.716

In [34]:
import json

with open('./template.geojson', 'r') as template:
    template = json.load(template)
    template['features'] = features
    template['minCrimeCount'] = int(crime_freq.min())
    template['maxCrimeCount'] = int(crime_freq.max())

with open('2018-11-greater-manchester-street.geojson', 'w') as output:
    json.dump(template, output)